In [1]:
#imports
import pandas as pd
import json
import numpy as np
from sklearn.neighbors import BallTree

In [2]:
#Define constants
distance_threshold_in_KM = 0.1
earth_radius = 6378
distance_threshold_in_rads = distance_threshold_in_KM / earth_radius

#load data and convert to radians
json_dataframe = pd.read_json("SensorData2.json")
csv_dataframe = pd.read_csv("SensorData1.csv")

coords_csv = csv_dataframe[['latitude', 'longitude']].values
coords_json = json_dataframe[['latitude', 'longitude']].values

coords_csv_radians = np.radians(coords_csv)
coords_json_radians = np.radians(coords_json)



In [3]:
#create ball tree of json with haversine
json_balltree = BallTree(coords_json_radians , metric="haversine")
#query csv data against json tree
distances, indices = json_balltree.query(coords_csv_radians, k=1)

matched_pairs = {}
#Go through distances and add to dictionary if in threshold
for i, distance in enumerate(distances):
    if distance <= distance_threshold_in_rads:
        csv_id = int(csv_dataframe.iloc[i]['id'])
        json_id = int(json_dataframe.iloc[indices[i][0]]['id'])
        matched_pairs[csv_id] = json_id

print(matched_pairs)

with open('output.json', 'w') as f:
    json.dump(matched_pairs, f)



{8: 12, 23: 40, 19: 13, 11: 37, 2: 9, 6: 19, 12: 17, 16: 33, 15: 23}
